## SCENIC.loom导入R里面进行可视化

In [ ]:
#加载R包：
rm(list=ls())
library(Seurat)
library(SCopeLoomR)
library(AUCell)
library(SCENIC)
library(dplyr)
library(KernSmooth)
library(RColorBrewer)
library(plotly)
library(BiocParallel)
library(grid)
library(ComplexHeatmap)
library(data.table)
# library(scRNAseq)
library(patchwork)
library(ggplot2) 
library(stringr)
library(circlize)
library(qs)

In [ ]:
source("../custom_seurat_functions.R")

### Step1. 提取 out_SCENIC.loom 信息

In [ ]:
#### 1.提取 out_SCENIC.loom 信息
loom <- open_loom('out_SCENIC.loom') 

regulons_incidMat <- get_regulons(loom, column.attr.name="Regulons")
regulons_incidMat[1:4,1:4] 

In [ ]:
regulons <- regulonsToGeneLists(regulons_incidMat)
regulonAUC <- get_regulons_AUC(loom,column.attr.name='RegulonsAUC')
regulonAucThresholds <- get_regulon_thresholds(loom)
tail(regulonAucThresholds[order(as.numeric(names(regulonAucThresholds)))])

In [ ]:
embeddings <- get_embeddings(loom)  
close_loom(loom)

rownames(regulonAUC)

### Step2. 加载Seurat Data

In [ ]:
# Macrophages <- qread("/home/guoliming/Brown/ALI_CRL/ALI/Macrophages/Outdata/Sub_annotion.qs")
# seurat.data  <- subset(Macrophages, subset = celltype %in% c("Macrophages_c2","Macrophages_c9","Macrophages_c10"))
# seurat.data <- subset(Macrophages, subset = !grepl("\\(", celltype))

seurat.data <- qread("/home/guoliming/Brown/ALI_Gaoji/sc_bulk_analysis_Epithelial/7_RNAseq/all_cells.qs")


In [ ]:
# qsave(seurat.data,file="Macro_Sub_annotion.qs")

In [ ]:
### 2.加载SeuratData
# seurat.data = qread(file = "./Outdata/Step1.pySCENIC_test_seurat.qs")
#seurat.data = readRDS("./pbmc3k.test.seurat.Rds")
# seurat.data <- seurat.data %>% NormalizeData(verbose = F) %>%
#   FindVariableFeatures(selection.method = "vst", nfeatures = 2000, verbose = F) %>% 
#   ScaleData(verbose = F) %>%
#   RunPCA(npcs = 30, verbose = F)

# n.pcs = 30
# seurat.data <- seurat.data %>% 
#   RunUMAP(reduction = "pca", dims = 1:n.pcs, verbose = F) %>% 
#   FindNeighbors(reduction = "pca", k.param = 10, dims = 1:n.pcs)

In [ ]:
# 这里有自带的注释
# seurat.data$seurat_annotations[is.na(seurat.data$seurat_annotations)] = "B"
# Idents(seurat.data) <- "seurat_annotations"

# options(repr.plot.width = 6, repr.plot.height = 4.5)
# DimPlot(seurat.data, reduction = "umap", label=T) 

In [ ]:
## 把SCENIC的结果与seurat数据合并
sub_regulonAUC <- regulonAUC[,match(colnames(seurat.data),colnames(regulonAUC))]
#确认是否一致
identical(colnames(sub_regulonAUC), colnames(seurat.data))
dim(sub_regulonAUC)

In [ ]:
cellTypes <- data.frame(row.names = colnames(seurat.data), 
                        celltype = seurat.data$celltype)
head(cellTypes)

In [ ]:
sub_regulonAUC[1:4,1:4] 

In [ ]:
library(SummarizedExperiment)

In [ ]:
scenic_res = assay(sub_regulonAUC) %>% as.matrix()
seurat.data[["scenic"]] <- SeuratObject::CreateAssayObject(counts = scenic_res)
seurat.data <- SeuratObject::SetAssayData(seurat.data, slot = "scale.data",
                                  new.data = scenic_res, assay = "scenic")

In [ ]:
seurat.data

## 保存Tf-target的对应关系

In [ ]:
tf_targets <- tibble(
  TF = rep(names(regulons), sapply(regulons, length)),  # TF 名称重复
  TargetGene = unlist(regulons)                        # 对应靶基因
)

# 导出为 CSV
write.csv(tf_targets, "SCENIC_TF_Targets.csv", row.names = FALSE)

## 得到细胞亚群特异性TF

In [ ]:
### 4.1. TF活性均值
# 看看不同单细胞亚群的转录因子活性平均值
# Split the cells by cluster:
selectedResolution <- "celltype" # select resolution
cellsPerGroup <- split(rownames(cellTypes), 
                       cellTypes[,selectedResolution])

# 去除extened regulons
sub_regulonAUC <- sub_regulonAUC[onlyNonDuplicatedExtended(rownames(sub_regulonAUC)),] 
dim(sub_regulonAUC)

# Calculate average expression:
regulonActivity_byGroup <- sapply(cellsPerGroup,
                                  function(cells) 
                                    rowMeans(getAUC(sub_regulonAUC)[,cells]))

# Scale expression. 
# Scale函数是对列进行归一化，所以要把regulonActivity_byGroup转置成细胞为行，基因为列
# 参考：https://www.jianshu.com/p/115d07af3029
regulonActivity_byGroup_Scaled <- t(scale(t(regulonActivity_byGroup),
                                          center = T, scale=T)) 
# 同一个regulon在不同cluster的scale处理
dim(regulonActivity_byGroup_Scaled)
regulonActivity_byGroup_Scaled=na.omit(regulonActivity_byGroup_Scaled)

## 可视化fc>3的

In [ ]:
# ### 4.3 其他查看TF方式
# rss=regulonActivity_byGroup_Scaled
# head(rss)
# df = do.call(rbind,
#              lapply(1:ncol(rss), function(i){
#                dat = data.frame(
#                  path = rownames(rss),
#                  cluster = colnames(rss)[i],
#                  sd.1 = rss[,i],
#                  sd.2 = apply(rss[,-i], 1, median)  
#                )
#              }))
# df$fc = df$sd.1 - df$sd.2

# ## 可视化fc>3的

# Macrophages_c2_tf <- df %>%
#   filter(cluster %in% c("Macrophages_c2","Macrophages_c10","Macrophages_c9")) %>% 
#   filter(fc > 3) %>% 
#   # top_n(5, fc) %>% 
# arrange(desc(fc))

# # top5 <- Macrophages_c2_tf %>% group_by(cluster) %>% top_n(5, fc)
# rowcn = data.frame(path = Macrophages_c2_tf$cluster) 
# n = rss[Macrophages_c2_tf$path,] 
# #rownames(rowcn) = rownames(n)

# options(repr.plot.width = 12, repr.plot.height = 8)

# pdf("pheatmap_fc3.pdf", width=12, height=8)
# pheatmap(n,
#          annotation_row = rowcn,
#          show_rownames = T,
#          fontsize = 15)
# dev.off()

## 可视化top3的

In [ ]:
### 4.3 其他查看TF方式
rss=regulonActivity_byGroup_Scaled
# head(rss)
df = do.call(rbind,
             lapply(1:ncol(rss), function(i){
               dat = data.frame(
                 path = rownames(rss),
                 cluster = colnames(rss)[i],
                 sd.1 = rss[,i],
                 sd.2 = apply(rss[,-i], 1, median)  
               )
             }))
df$fc = df$sd.1 - df$sd.2

## 可视化top5的

Macrophages_c2_tf <- df %>%
  filter(cluster %in% c("Macrophages_c2","Macrophages_c10","Macrophages_c9")) %>% 
  # filter(fc > 3) %>% 
  # top_n(5, fc) %>% 
arrange(desc(fc))

Macrophages_c2_tf <- Macrophages_c2_tf %>% group_by(cluster) %>% top_n(3, fc)
rowcn = data.frame(path = Macrophages_c2_tf$cluster) 
n = rss[Macrophages_c2_tf$path,] 
#rownames(rowcn) = rownames(n)

# top5
options(repr.plot.width = 12, repr.plot.height = 6)

pdf("pheatmap_top3_.pdf", width=12, height=6)
pheatmap(n,
         annotation_row = rowcn,
         show_rownames = T,
         fontsize = 15)
dev.off()
         

In [ ]:
unique(Macrophages_c2_tf$cluster)

In [ ]:
write.csv(Macrophages_c2_tf,
          file = "Macrophages_tf_Top3.csv",
          row.names = FALSE)   # 不要行名

## 可选-bulk-sigmarker & scenic调控基因求交集（这部分在6_Bulk_RNAseq可做）

In [ ]:
# 加载包
library(dplyr)

# SCENIC_TF_Targets为TF-Target对应表格
scenic_df <- read.csv("/home/guoliming/Brown/ALI_Gaoji/sc_bulk_analysis_Epithelial/8_scenic/SCENIC_TF_Targets.csv")

### common_genes_Log2FC_Padj为bulk-RNA差异表达基因和逆时序分析的交集
rna_df <- read.csv("/home/guoliming/Brown/ALI_Gaoji/sc_bulk_analysis_Epithelial/7_RNAseq/c2_common_gene_Log2FC_2.csv")

# 确保列名一致
colnames(scenic_df)[1:2]  # 应该是 "TF" 和 "TargetGene"
colnames(rna_df)[1]       # 应该是 "Gene_name"

In [ ]:
Macrophages_c2_tf

In [ ]:
Macrophages_c2_tf_gene <- Macrophages_c2_tf %>%
  filter(cluster == "Macrophages_c2")

scenic_filtered <- scenic_df %>%
  filter(TF %in% Macrophages_c2_tf_gene$path)

In [ ]:
scenic_filtered
write.csv(scenic_filtered,
          file = "c2_scenic_gene_target.csv",
          row.names = FALSE)   # 不要行名

In [ ]:
# 取交集
inter_genes <- intersect(unique(scenic_filtered$TargetGene),
                         rna_df$Gene_name)

# 转换为 data.frame
inter_genes_df <- data.frame(Gene = inter_genes)

# 写出到 CSV
write.csv(inter_genes_df, "c2___intersect_genes.csv", row.names = FALSE)


In [ ]:
inter_genes

## 可视化

In [ ]:
## 绘制单个基因在不同亚型的表达量
regulonsToPlot = c("Stat2(+)","Rreb1(+)","Stat1(+)")
regulonsToPlot %in% row.names(sub_regulonAUC)
# Vis
p1 = DotPlot(seurat.data, features = unique(regulonsToPlot), assay = "scenic") + coord_flip() + RotatedAxis()
p2 = VlnPlot(seurat.data, features = regulonsToPlot, assay = "scenic",pt.size = 0)&labs(y="TF activity")
p3 = RidgePlot(seurat.data, features = regulonsToPlot, assay = "scenic" , ncol = 2)&labs(x="TF activity")
# p4 = FeaturePlot(seurat.data,features = regulonsToPlot)

options(repr.plot.width = 15, repr.plot.height = 7)
wrap_plots(p1
# ,p2,p3
# ,p4
)